In [2]:
%pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
  Using cached dash-3.2.0-py3-none-any.whl.metadata (10 kB)
  Using cached ipywidgets-8.1.7-py3-none-any.whl.metadata (2.4 kB)
  Using cached jupyter-1.1.1-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached importlib_metadata-8.7.0-py3-none-any.whl.metadata (4.8 kB)
  Using cached retrying-1.4.2-py3-none-any.whl.metadata (5.5 kB)
  Using cached widgetsnbextension-4.0.14-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl.metadata (20 kB)
  Using cached notebook-7.4.5-py3-none-any.whl.metadata (10 kB)
  Using cached jupyter_console-6.6.3-py3-none-any.whl.metadata (5.8 kB)
  Using cached nbconvert-7.16.6-py3-none-any.whl.metadata (8.5 kB)
  Using cached jupyterlab-4.4.7-py3-none-any.whl.metadata (16 kB)
  Using cached zipp-3.23.0-py3-none-any.whl.metadata (3.6 kB)
  Using cached async_lru-2.0.5-py3-none-any.whl.metadata (4.5 kB)
  Using cached httpx-0.28.1-py3-none-an

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress t

In [3]:
# -------------------------------
# Imports
# -------------------------------

# Standard Library
import json
import os
import re
import threading
import time
from datetime import datetime

# Third-party packages
import dash
from dash import dcc, html
from dash.dependencies import Input as DashInput, Output
import ipywidgets as widgets
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import requests
from IPython.display import display, clear_output, HTML
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.layers import Conv1D, Dense, Dropout, Input, LSTM
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.optimizers import Adam


# -------------------------------
# Settings laden / default erstellen
# -------------------------------
settings_file = "settings.json"
if not os.path.exists(settings_file):
    default_settings = {
        "allgemein_settings": {
            "model_folder": "models",
            "use_model_file": "Heusc_v0.3_20250913_024701.keras",
            "sequence_length": 60,
            "train_epochs": 10000,
            "batch_size": 32,
            "validation_split": 0.2,
            "early_stopping_patience": 5,
            "feature_scaling": True,
            "use_dropout": True,
            "dropout_rate": 0.2,
            "loss_function": "binary_crossentropy",
            "optimizer": "adam",
            "debug_mode": True
        },
        "offline": {
            "enabled": True,
            "csv_folder": "csv/binance",
            "csv_file": "BTCUSDT-1m-1y-binance-2025-09-12_22-40-01.csv"
        },
        "online": {
            "symbols": ["BTC","ETH","SOL"],
            "currencies": ["USDT","USDT","USDT"],
            "interval": "1m",
            "poll_seconds": 10,
            "max_live_train_minutes": 60
        },
        "balance": {
            "use_simulation": True,
            "initial_balance": 1000,
            "save_on_profit_percent": 50,
            "game_over_threshold": 0,
            "balance_reward_factor": 1.0,
            "retrain_from_best": True
        },
        "training_options": {
            "use_weighting": True,
            "punish_on_wrong": True,
            "predict_confidence": True,
            "log_transactions": True,
            "continual_learning": True,
            "transfer_learning": True,
            "ensemble_enabled": True
        }
    }
    with open(settings_file,"w") as f:
        json.dump(default_settings, f, indent=4)

with open(settings_file,"r") as f:
    settings = json.load(f)

allg = settings['allgemein_settings']
online = settings['online']
offline = settings['offline']
balance_settings = settings['balance']
training_options = settings['training_options']

print(f"allg: {allg}")
print(f"online: {online}")
print(f"offline: {offline}")
print(f"balance_settings: {balance_settings}")
print(f"training_opts: {training_options}")



allg: {'model_folder': 'models', 'use_model_file': '', 'sequence_length': 128, 'train_epochs': 1000, 'batch_size': 32, 'validation_split': 0.2, 'early_stopping_patience': 5, 'feature_scaling': True, 'use_dropout': True, 'dropout_rate': 0.2, 'loss_function': 'binary_crossentropy', 'optimizer': 'adam', 'debug_mode': True}
online: {'symbols': ['BTC', 'ETH', 'SOL'], 'currencies': ['USDT', 'USDT', 'USDT'], 'interval': '1m', 'poll_seconds': 10, 'max_live_train_minutes': 60}
offline: {'enabled': True, 'csv_folder': 'csv/binance', 'csv_file': 'BTCUSDT-1m-1mo-binance-2025-09-12_22-43-22.csv'}
balance_settings: {'use_simulation': True, 'initial_balance': 1000, 'save_on_profit_percent': 50, 'game_over_threshold': 0, 'balance_reward_factor': 1.0, 'retrain_from_best': True}
training_opts: {'use_weighting': True, 'punish_on_wrong': True, 'predict_confidence': True, 'log_transactions': True, 'continual_learning': True, 'transfer_learning': True, 'ensemble_enabled': True}


In [ ]:
# Startordner für CSVs (relativer Pfad)
notebook_dir = os.getcwd()
start_folder = os.path.abspath(os.path.join(notebook_dir, f"../../{settings['offline']['csv_folder']}"))
os.makedirs(start_folder, exist_ok=True)

# Widgets
folder_dropdown = widgets.Dropdown(description="Ordner:", layout=widgets.Layout(width="70%"))
file_dropdown = widgets.Dropdown(description="Datei:", layout=widgets.Layout(width="70%"))
save_button = widgets.Button(description="Speichern", button_style="success")
output = widgets.Output()

def update_folder_options(folder):
    items = [d for d in os.listdir(folder) if os.path.isdir(os.path.join(folder, d))]
    folder_dropdown.options = [".."] + items
    folder_dropdown.value = None
    
    files = [f for f in os.listdir(folder) if f.endswith(".csv")]
    if not files:
        files = ["<keine CSV gefunden>"]
    file_dropdown.options = files
    if settings['offline'].get('csv_file') in files:
        file_dropdown.value = settings['offline']['csv_file']
    else:
        file_dropdown.value = files[0]

def folder_changed(change):
    if change['new'] is not None:
        selected = change['new']
        new_folder = os.path.normpath(os.path.join(current_folder[0], ".." if selected==".." else selected))
        if os.path.exists(new_folder):
            current_folder[0] = new_folder
            update_folder_options(new_folder)
            with output:
                clear_output()
                print(f"[INFO] Aktueller Ordner: {new_folder}")
                print(f"[INFO] Aktuelle CSV: {file_dropdown.value}")

def save_csv(b):
    selected_file = file_dropdown.value
    if selected_file != "<keine CSV gefunden>":
        settings['offline']['csv_file'] = selected_file
        with open(settings_file, "w") as f:
            json.dump(settings, f, indent=4)
        with output:
            clear_output()
            print(f"[INFO] CSV-Datei in settings.json auf '{selected_file}' gesetzt")
    else:
        with output:
            clear_output()
            print("[WARN] Keine CSV-Datei zum Speichern vorhanden!")

current_folder = [start_folder]
update_folder_options(start_folder)

folder_dropdown.observe(folder_changed, names='value')
save_button.on_click(save_csv)

display(folder_dropdown, file_dropdown, save_button, output)


Dropdown(description='Ordner:', layout=Layout(width='70%'), options=('..',), value=None)

Dropdown(description='Datei:', index=3, layout=Layout(width='70%'), options=('BTCUSDT-1m-3mo-binance-2025-09-1…

Button(button_style='success', description='Speichern', style=ButtonStyle())

Output()

In [ ]:
# Absoluten Pfad zur CSV-Datei erstellen
csv_folder = os.path.abspath(os.path.join(notebook_dir, f"../../{offline['csv_folder']}"))
csv_file = offline['csv_file']  # z.B. 'BTCUSDT-1m-1y-binance-2025-09-13_11-37-53.csv'

csv_file_path = os.path.join(csv_folder, csv_file)

print(f"Pfad zur CSV: {csv_file_path}")
print(f"Existiert Datei? {os.path.exists(csv_file_path)}")  # True = OK

def load_csv_data(file_path, sequence_length=60, feature_scaling=True):
    # CSV laden
    df = pd.read_csv(file_path)
    
    # Timestamp in datetime umwandeln
    if 'timestamp' not in df.columns:
        raise ValueError("CSV enthält keine 'timestamp'-Spalte!")
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df.sort_values('timestamp', inplace=True)

    # Wichtige Features auswählen
    required_features = ['open', 'high', 'low', 'close', 'volume']
    missing_features = [f for f in required_features if f not in df.columns]
    if missing_features:
        raise ValueError(f"CSV fehlt benötigte Spalten: {missing_features}")
    
    # Fehlende Werte entfernen
    df = df.dropna(subset=required_features)
    
    data = df[required_features].values
    
    # Feature Scaling optional
    scaler = None
    if feature_scaling:
        scaler = StandardScaler()
        data = scaler.fit_transform(data)

    # Sequenzen und Labels erstellen
    sequences, labels = [], []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i+sequence_length])
        # Label: 1 wenn Close der nächsten Kerze steigt, sonst 0
        labels.append(1 if data[i+sequence_length][3] > data[i+sequence_length-1][3] else 0)

    sequences = np.array(sequences)
    labels = np.array(labels)

    return sequences, labels, scaler

# Beispiel Nutzung
if offline['enabled']:
    csv_folder = os.path.abspath(os.path.join(notebook_dir, f"../../{offline['csv_folder']}"))
    csv_file = offline['csv_file']
    csv_file_path = os.path.join(csv_folder, csv_file)

    if os.path.exists(csv_file_path):
        seq_len = allg['sequence_length']
        X, y, scaler = load_csv_data(csv_file_path, sequence_length=seq_len, feature_scaling=allg['feature_scaling'])
        print(f"Data shape: X={X.shape}, y={y.shape}")
    else:
        print("[ERROR] CSV-Datei nicht gefunden! Bitte Pfad überprüfen.")

In [ ]:
# -------------------------------
# Buttons zum Speichern vorbereiten
# -------------------------------
def save_model(model, symbols="BTC-ETH-SOL", model_type="mini"):
    from datetime import datetime
    import os
    model_folder = "models"
    os.makedirs(model_folder, exist_ok=True)
    
    # Layer-Info
    layers = []
    for layer in model.layers:
        if isinstance(layer, tf.keras.layers.Conv1D):
            layers.append(f"C{layer.filters}k{layer.kernel_size[0]}_{layer.activation.__name__}")
        elif isinstance(layer, tf.keras.layers.LSTM):
            layers.append(f"L{layer.units}_{layer.activation.__name__}")
        elif isinstance(layer, tf.keras.layers.Dense):
            layers.append(f"D{layer.units}_{layer.activation.__name__}")
    
    layer_str = "_".join(layers)
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    model_name = f"Heusc_v_{model_type}_{symbols}_{layer_str}_{timestamp}.keras"
    model_path = os.path.join(model_folder, model_name)
    model.save(model_path)
    print(f"Model saved: {model_path}")

def on_save_mini(b):
    save_model(mini_model, model_type="mini")
def on_save_big(b):
    save_model(big_model, model_type="big")

save_mini_btn = widgets.Button(description="Save Mini Model")
save_mini_btn.on_click(on_save_mini)
save_big_btn = widgets.Button(description="Save Big Model")
save_big_btn.on_click(on_save_big)
display(save_mini_btn, save_big_btn)

In [ ]:
# -------------------------------
# Trainingsroutine parallel
# -------------------------------
def train_mini():
    global mini_model
    mini_model = create_small_cnn_lstm(seq_len, n_features)
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    mini_model.fit(
        X, y,
        epochs=1000,
        batch_size=32,
        validation_split=0.2,
        callbacks=[early_stop],
        verbose=1
    )

def train_big():
    global big_model
    big_model = create_big_cnn_lstm(seq_len, n_features)
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    big_model.fit(
        X, y,
        epochs=1000,
        batch_size=32,
        validation_split=0.2,
        callbacks=[early_stop],
        verbose=1
    )

# Threads starten
thread_mini = threading.Thread(target=train_mini)
thread_big = threading.Thread(target=train_big)
thread_mini.start()
thread_big.start()